In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
from unidecode import unidecode

In [3]:
from config import spark

22/03/18 09:45:47 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/03/18 09:45:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/18 09:45:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/18 09:45:48 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
HD = '/media/fabio/19940C2755DB566F/'
DATALAKE = 'PAMepi/datalake/'
RAW = 'raw_data_covid19_version-2022-02-23/IRECE'

In [5]:
df = spark.read.csv(HD + DATALAKE + RAW, header=True, sep=';')

In [53]:
df.count()

114889

In [13]:
df.columns

['paciente_id',
 'paciente_idade',
 'paciente_enumSexoBiologico',
 'paciente_racaCor_valor',
 'paciente_endereco_coIbgeMunicipio',
 'paciente_endereco_nmMunicipio',
 'paciente_endereco_uf',
 'vacina_dataAplicacao',
 'vacina_descricao_dose',
 'vacina_nome']

In [15]:
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

In [17]:
df = df.withColumnRenamed('paciente_id', 'id') \
    .withColumnRenamed('paciente_idade', 'idade') \
    .withColumnRenamed('paciente_enumsexobiologico', 'sexo') \
    .withColumnRenamed('paciente_racacor_valor', 'raca') \
    .withColumnRenamed('paciente_endereco_coibgemunicipio', 'mun_res') \
    .withColumnRenamed('paciente_endereco_nmmunicipio', 'nome_mun_res') \
    .withColumnRenamed('paciente_endereco_uf', 'uf_res') \
    .withColumnRenamed('vacina_dataaplicacao', 'data') \
    .withColumnRenamed('vacina_descricao_dose', 'dose') \
    .withColumnRenamed('vacina_nome', 'vacina')

In [18]:
columns = [
    'data', 'id', 'idade', 'sexo', 'raca', 'uf_res', 'nome_mun_res',
    'mun_res', 'vacina', 'dose'
]

In [19]:
df.select(columns).show()

+----------+--------------------+-----+----+--------------+------+------------+-------+--------------------+-------+
|      data|                  id|idade|sexo|          raca|uf_res|nome_mun_res|mun_res|              vacina|   dose|
+----------+--------------------+-----+----+--------------+------+------------+-------+--------------------+-------+
|2021-07-06|8b5dcadea57759162...|   50|   M|SEM INFORMACAO|    BA|       IRECE| 291460|COVID-19 PFIZER -...|1ª Dose|
|2021-09-20|f9ee5ff50c418dd49...|   44|   M|       AMARELA|    BA|       IRECE| 291460|COVID-19 PFIZER -...|2ª Dose|
|2021-06-28|51a9e30784797576f...|   59|   M|        BRANCA|    BA|       IRECE| 291460|COVID-19 PFIZER -...|1ª Dose|
|2021-06-16|1b49149a9136191a6...|   52|   M|       AMARELA|    BA|       IRECE| 291460|COVID-19 ASTRAZEN...|1ª Dose|
|2021-08-16|d48d645a0e39beced...|   28|   F|       AMARELA|    BA|       IRECE| 291460|COVID-19 PFIZER -...|1ª Dose|
|2021-10-14|cc3198945e285ebe1...|   39|   M|       AMARELA|    B

In [194]:
unidecode_udf = F.udf(
    lambda string: unidecode(string).lower() if string is not None else None,
    T.StringType()
)

In [195]:
df = df.withColumn('sexo', unidecode_udf('sexo')) \
    .withColumn('uf_res', unidecode_udf('uf_res')) \
    .withColumn('nome_mun_res', unidecode_udf('nome_mun_res')) \
    .withColumn('vacina', unidecode_udf('vacina')) \
    .withColumn('dose', unidecode_udf('dose')) \
    .withColumn('raca', unidecode_udf('raca'))

In [196]:
df.select(columns).show()

+----------+--------------------+-----+----+--------------+------+------------+-------+------+-------+
|      data|                  id|idade|sexo|          raca|uf_res|nome_mun_res|mun_res|vacina|   dose|
+----------+--------------------+-----+----+--------------+------+------------+-------+------+-------+
|2021-07-06|8b5dcadea57759162...|   50|   m|sem informacao|    ba|       irece| 291460|     3|1a dose|
|2021-09-20|f9ee5ff50c418dd49...|   44|   m|       amarela|    ba|       irece| 291460|     3|2a dose|
|2021-06-28|51a9e30784797576f...|   59|   m|        branca|    ba|       irece| 291460|     3|1a dose|
|2021-06-16|1b49149a9136191a6...|   52|   m|       amarela|    ba|       irece| 291460|     1|1a dose|
|2021-08-16|d48d645a0e39beced...|   28|   f|       amarela|    ba|       irece| 291460|     3|1a dose|
|2021-10-14|cc3198945e285ebe1...|   39|   m|       amarela|    ba|       irece| 291460|     1|2a dose|
|2021-07-20|dd4b1bb12b1a44632...|   40|   m|       amarela|    ba|       

In [197]:
df.select('vacina').distinct().show(truncate=False)

+------+
|vacina|
+------+
|3     |
|1     |
|4     |
|2     |
+------+



In [198]:
df.select('data', 'vacina', 'dose').where(df.dose == 'unica').show()

+----------+------+-----+
|      data|vacina| dose|
+----------+------+-----+
|2021-07-08|     4|unica|
|2021-06-28|     4|unica|
|2021-06-28|     4|unica|
|2021-07-14|     4|unica|
|2021-10-14|     4|unica|
|2021-07-07|     4|unica|
+----------+------+-----+



In [199]:
df = df.withColumn(
    'vacina',
    F.when(
        F.col('vacina').contains('astrazeneca'),
        F.lit(1)
    ).when(
        F.col('vacina').contains('coronavac'),
        F.lit(2)
    ).when(
        F.col('vacina').contains('pfizer'),
        F.lit(3)
    ).when(
        F.col('vacina').contains('janssen'),
        F.lit(4)
    ).otherwise(F.col('vacina'))
)

In [200]:
df = df.filter(df.data <= '2021-11-15')

In [ ]:
# df.select(columns) \
# .write.csv(HD + DATALAKE + '/vacinacao_paper', header=True)

In [188]:
# df = spark.read.csv(HD + DATALAKE + '/vacinacao_paper', header=True)

In [201]:
d1 = df.groupby('id').pivot('dose') \
.agg(F.concat_ws(',', F.first(F.col('data')), F.first(F.col('vacina'))))

d2 = df.groupby('id').agg(F.collect_set(F.col('sexo')).alias('sexo'))

d3 = df.groupby('id').agg(F.collect_set(F.col('raca')).alias('raca'))

d4 = df.groupby('id').agg(F.collect_set(F.col('idade')).alias('idade'))

d5 = df.groupby('id').agg(F.collect_set(F.col('mun_res')).alias('mun_res'))

In [198]:
# d1.write.csv('doses', header=True, compression='gzip')

# d2.withColumn('sexo', F.col('sexo').cast(T.StringType())) \
# .write.csv('sexo', header=True, compression='gzip')

# d3.withColumn('raca', F.col('raca').cast(T.StringType())) \
# .write.csv('raca', header=True, compression='gzip')

# d4.withColumn('idade', F.col('idade').cast(T.StringType())) \
# .write.csv('idade', header=True, compression='gzip')

# d5.withColumn('mun_res', F.col('mun_res').cast(T.StringType())) \
# .write.csv('mun_res', header=True, compression='gzip')

In [202]:
df1 = d1.join(d2, on=['id']).join(d3, on=['id']).join(d4, on=['id']).join(d5, on=['id'])

In [203]:
df1.count()

65287

In [210]:
df1.show(1, vertical=True)

-RECORD 0------------------------------
 id             | 0000261e4a37da441... 
 1a dose        | 2021-08-19,3         
 2a dose        |                      
 3a dose        |                      
 dose           |                      
 dose adicional |                      
 reforco        |                      
 unica          |                      
 sexo           | [f]                  
 raca           | [parda]              
 idade          | [25]                 
 mun_res        | [291460]             
only showing top 1 row



In [211]:
df1 = df1.withColumn('sexo_n', F.size('sexo')) \
        .withColumn('raca_n', F.size('raca')) \
        .withColumn('idade_n', F.size('idade')) \
        .withColumn('mun_res_n', F.size('mun_res'))

In [48]:
df1.select('sexo_n').distinct().show()

+------+
|sexo_n|
+------+
|     1|
|     2|
+------+



In [49]:
df1.select('raca_n').distinct().show()


+------+
|raca_n|
+------+
|     1|
|     2|
+------+



In [50]:
df1.select('idade_n').distinct().show()


+-------+
|idade_n|
+-------+
|      1|
|      2|
+-------+



In [51]:
df1.select('mun_res_n').distinct().show()

+---------+
|mun_res_n|
+---------+
|        1|
+---------+



In [212]:
@F.udf()
def fx_etaria(idade):
    try:
        if (idade >= 0 and idade < 18):
            return '0-17'
        elif (idade >= 18 and idade < 30):
            return '18-29'
        elif (idade >= 30 and idade < 40):
            return '30-39'
        elif (idade >= 40 and idade < 50):
            return '40-49'
        elif (idade >= 50 and idade < 65):
            return '50-64'
        elif (idade >= 65 and idade < 75):
            return '65-74'
        elif (idade >= 75 and idade < 85):
            return '75-84'
        else:
            return '85+'
    except TypeError:
        None

In [55]:
###### IRECE ######

In [ ]:
# sexo

In [94]:
df1.withColumn('sexo', F.col('sexo').cast(T.StringType())) \
    .withColumn('m', F.when(F.col('sexo').contains('m'), F.lit(1)).otherwise(F.lit(0))) \
    .withColumn('f', F.when(F.col('sexo').contains('f'), F.lit(1)).otherwise(F.lit(0))) \
    .withColumn('i', F.when(F.col('sexo').contains('i'), F.lit(1)).otherwise(F.lit(0))) \
    .groupby('f', 'm', 'i').count().coalesce(1).write.csv(HD + DATALAKE + '/irece_sexo', header=True)

In [95]:
# raca

In [104]:
df1.withColumn('idade', F.col('idade').cast(T.StringType())) \
    .withColumn('idade_c', F.regexp_extract(F.col('idade'), r'(\d+)', 1)) \
    .withColumn('fx_etaria', fx_etaria(F.col('idade_c').cast(T.IntegerType()))) \
    .groupby('fx_etaria').count().coalesce(1).write.csv(HD + DATALAKE + '/irece_fxetaria', header=True)

In [106]:
# doses

In [297]:
doses = ['1a dose', '2a dose', '3a dose', 'dose', 'dose adicional', 'reforco', 'unica']

In [298]:
for dose in doses:
    df1 = df1.withColumn(dose, F.when(F.length(F.col(dose)) == 0, None).otherwise(F.col(dose)))
    df1 = df1.withColumn(dose, F.split(F.col(dose), ',').getItem(0))


In [299]:
df2 = adjust_dose_date(df1)

In [300]:
df2 = generate_labels_dose(df2)

In [301]:
df2.select(*doses, 'prim_dose', 'vac_s_ref', 'vac_c_ref').show()

+----------+----------+-------+----+--------------+-------+-----+----------+----------+---------+
|   1a dose|   2a dose|3a dose|dose|dose adicional|reforco|unica| prim_dose| vac_s_ref|vac_c_ref|
+----------+----------+-------+----+--------------+-------+-----+----------+----------+---------+
|2021-08-19|      null|   null|null|          null|   null| null|2021-08-19|      null|     null|
|2021-10-22|      null|   null|null|          null|   null| null|2021-10-22|      null|     null|
|2021-08-19|2021-09-22|   null|null|          null|   null| null|      null|2021-09-22|     null|
|2021-08-17|2021-11-11|   null|null|          null|   null| null|      null|2021-11-11|     null|
|2021-10-21|      null|   null|null|          null|   null| null|2021-10-21|      null|     null|
|2021-03-24|2021-04-22|   null|null|          null|   null| null|      null|2021-04-22|     null|
|2021-06-23|2021-09-20|   null|null|          null|   null| null|      null|2021-09-20|     null|
|2021-07-20|2021-11-

In [302]:
df2.groupby('prim_dose', 'vac_s_ref', 'vac_c_ref').count().coalesce(1) \
.write.csv(HD + DATALAKE + '/irece_ciclo_vacinal', mode='overwrite', header=True)

In [306]:
df2 = df2.withColumn('dt_pattern', F.lit('2021-11-15'))

In [309]:
df2 = df2.withColumn(
    'i >= 6',
    F.when(
        F.col('prim_dose').isNotNull(),
        F.when(F.months_between(
            F.col('dt_pattern'),
            F.col('prim_dose')
        ) > 6, 1)
    )
).withColumn(
    'i >= 2 < 6',
    F.when(
        F.col('prim_dose').isNotNull(),
        F.when(
            (F.months_between(
                F.col('dt_pattern'),
                F.col('prim_dose')) >= 2
             ) &
            (F.months_between(
                F.col('dt_pattern'),
                F.col('prim_dose')) < 6
             ), 1)
    )
).withColumn(
    'i < 2',
    F.when(
        F.col('prim_dose').isNotNull(),
        F.when(
            F.months_between(
                F.col('dt_pattern'),
                F.col('prim_dose')
            ) < 2, 1
        )
    )
)

In [314]:
df2= df2.withColumn(
    'c >= 6',
    F.when(
        F.col('vac_s_ref').isNotNull(),
        F.when(
            F.months_between(
                F.col('dt_pattern'),
                F.col('vac_s_ref')
            ) >= 6, 1
        )
    )
)

df2 = df2.withColumn(
    'c >= 2 < 6',
    F.when(
        F.col('vac_s_ref').isNotNull(),
        F.when(
            (F.months_between(
                F.col('dt_pattern'),
                F.col('vac_s_ref')) >= 2
             ) &
            (F.months_between(
                F.col('dt_pattern'),
                F.col('vac_s_ref')) < 6
             ), 1
        )
    )
)


df2 = df2.withColumn(
    'c < 2',
    F.when(
        F.col('vac_s_ref').isNotNull(),
        F.when(
            F.months_between(
                F.col('dt_pattern'),
                F.col('vac_s_ref')
            ) < 2, 1
        )
    )
)

In [315]:
df2= df2.withColumn(
    'cb >= 6',
    F.when(
        F.col('vac_c_ref').isNotNull(),
        F.when(
            F.months_between(
                F.col('dt_pattern'),
                F.col('vac_c_ref')
            ) >= 6, 1
        )
    )
)

df2 = df2.withColumn(
    'cb >= 2 < 6',
    F.when(
        F.col('vac_c_ref').isNotNull(),
        F.when(
            (F.months_between(
                F.col('dt_pattern'),
                F.col('vac_c_ref')) >= 2
             ) &
            (F.months_between(
                F.col('dt_pattern'),
                F.col('vac_c_ref')) < 6
             ), 1
        )
    )
)


df2 = df2.withColumn(
    'cb < 2',
    F.when(
        F.col('vac_c_ref').isNotNull(),
        F.when(
            F.months_between(
                F.col('dt_pattern'),
                F.col('vac_c_ref')
            ) < 2, 1
        )
    )
)

In [352]:
df2.withColumn('n prim_dose', F.when(F.col('prim_dose').isNotNull(), 1)) \
    .withColumn('n vac_s_ref', F.when(F.col('vac_s_ref').isNotNull(), 1)) \
    .withColumn('n vac_c_ref', F.when(F.col('vac_c_ref').isNotNull(), 1)) \
.agg(
    F.concat(
        F.count('n prim_dose'),
        F.lit(' ('),
        F.round((100 * F.count('n prim_dose')) / 65287, 2),
        F.lit('%)')
    ).alias('apenas primeira dose'),
    F.concat(
        F.count('n vac_s_ref'), 
        F.lit(' ('),
        F.round((100 * F.count('n vac_s_ref')) / 65287, 2),
        F.lit('%)')
    ).alias('completa sem reforço'),
    F.concat(
        F.count('n vac_c_ref'),
        F.lit(' ('),
        F.round((100 * F.count('n vac_c_ref')) / 65287, 2),
        F.lit('%)')
    ).alias('completa com reforço')
).show()

+--------------------+--------------------+--------------------+
|apenas primeira dose|completa sem reforço|completa com reforço|
+--------------------+--------------------+--------------------+
|      16792 (25.72%)|       44916 (68.8%)|        2470 (3.78%)|
+--------------------+--------------------+--------------------+



In [331]:
df2.withColumn('n prim_dose', F.when(F.col('prim_dose').isNotNull(), 1)) \
    .withColumn('n vac_s_ref', F.when(F.col('vac_s_ref').isNotNull(), 1)) \
    .withColumn('n vac_c_ref', F.when(F.col('vac_c_ref').isNotNull(), 1)) \
.groupby('n prim_dose', 'n vac_s_ref', 'n vac_c_ref').count().show()

+-----------+-----------+-----------+-----+
|n prim_dose|n vac_s_ref|n vac_c_ref|count|
+-----------+-----------+-----------+-----+
|       null|       null|       null| 1109|
|          1|       null|       null|16792|
|       null|          1|       null|44916|
|       null|       null|          1| 2470|
+-----------+-----------+-----------+-----+



In [359]:
df2.agg(
    F.concat(
        F.count('i >= 6'),
        F.lit(' ('),
        F.round((100 * F.count('i >= 6')) / 65287, 2),
        F.lit('%)')
    ).alias('i >= 6'),
    
    F.concat(
        F.count('i >= 2 < 6'),
        F.lit(' ('),
        F.round((100 * F.count('i >= 2 < 6')) / 65287, 2),
        F.lit('%)')
    ).alias('i >= 2 < 6'),
    
    F.concat(
        F.count('i < 2'),
        F.lit(' ('),
        F.round((100 * F.count('i < 2')) / 65287, 2),
        F.lit('%)')
    ).alias('i < 2'),
    
    F.concat(
        F.count('c >= 6'),
        F.lit(' ('),
        F.round((100 * F.count('c >= 6')) / 65287, 2),
        F.lit('%)')
    ).alias('c >= 6'),
    
    F.concat(
        F.count('c >= 2 < 6'),
        F.lit(' ('),
        F.round((100 * F.count('c >= 2 < 6')) / 65287, 2),
        F.lit('%)')
    ).alias('c >= 2 < 6'),
    
    F.concat(
        F.count('c < 2'),
        F.lit(' ('),
        F.round((100 * F.count('c < 2')) / 65287, 2),
        F.lit('%)')
    ).alias('c < 2'),
    
    F.concat(
        F.count('cb >= 6'),
        F.lit(' ('),
        F.round((100 * F.count('cb >= 6')) / 65287, 2),
        F.lit('%)')
    ).alias('cb >= 6'),
    
    F.concat(
        F.count('cb >= 2 < 6'),
        F.lit(' ('),
        F.round((100 * F.count('cb >= 2 < 6')) / 65287, 2),
        F.lit('%)')
    ).alias('cb >= 2 < 6'),
    
    F.concat(
        F.count('cb < 2'),
        F.lit(' ('),
        F.round((100 * F.count('cb < 2')) / 65287, 2),
        F.lit('%)')
    ).alias('cb < 2'),
    
    
).show(vertical=True, truncate=False)

-RECORD 0---------------------
 i >= 6      | 1083 (1.66%)   
 i >= 2 < 6  | 10107 (15.48%) 
 i < 2       | 5601 (8.58%)   
 c >= 6      | 4852 (7.43%)   
 c >= 2 < 6  | 20457 (31.33%) 
 c < 2       | 19607 (30.03%) 
 cb >= 6     | 1 (0.0%)       
 cb >= 2 < 6 | 58 (0.09%)     
 cb < 2      | 2411 (3.69%)   



In [319]:
df2.agg(
    F.count('i >= 6').astype('int').alias('i >= 6'),
    F.count('i >= 2 < 6').astype('int').alias('i >= 2 < 6'),
    F.count('i < 2').astype('int').alias('i < 2'),

    F.count('c >= 6').astype('int').alias('c >= 6'),
    F.count('c >= 2 < 6').astype('int').alias('c >= 2 < 6'),
    F.count('c < 2').astype('int').alias('c < 2'),

    F.count('cb >= 6').astype('int').alias('cb >= 6'),
    F.count('cb >= 2 < 6').astype('int').alias('cb >= 2 < 6'),
    F.count('cb < 2').astype('int').alias('cb < 2'),
).coalesce(1).write.csv(HD + DATALAKE + '/irece_ciclo_vacinal_tempo', mode='overwrite', header=True)

In [56]:
##### PAPER #####

In [120]:
df_sexo = spark.read.csv('sexo', header=True)

In [10]:
df_sexo.count()

161029567

In [121]:
df_sexo.show(5)

+--------------------+---------+
|                  id|     sexo|
+--------------------+---------+
|000009b2e3aaae800...|   [m, m]|
|00001904b82ec2994...|[m, m, m]|
|00002e4d7a02621aa...|[m, m, m]|
|000042056ba50ed6d...|   [f, f]|
|00004eadf6e493c07...|   [f, f]|
+--------------------+---------+
only showing top 5 rows



In [122]:
df_sexo = df_sexo.withColumn(
    'num_sexo',
    F.udf(len)(F.udf(set)(
        F.split(
            F.regexp_replace(
                F.col('sexo'), r'^\[|\]$', ''
            ), ', '
        )
    ))
)

In [301]:
df_sexo = df_sexo.withColumn(
    'sexo_set',
    F.udf(set)(
        F.split(
            F.regexp_replace(
                F.col('sexo'), r'^\[|\]$', ''
            ), ', ')
    )
)

In [124]:
df_sexo.show(5)

+--------------------+---------+--------+--------+
|                  id|     sexo|num_sexo|sexo_set|
+--------------------+---------+--------+--------+
|000009b2e3aaae800...|   [m, m]|       1|     [m]|
|00001904b82ec2994...|[m, m, m]|       1|     [m]|
|00002e4d7a02621aa...|[m, m, m]|       1|     [m]|
|000042056ba50ed6d...|   [f, f]|       1|     [f]|
|00004eadf6e493c07...|   [f, f]|       1|     [f]|
+--------------------+---------+--------+--------+
only showing top 5 rows



Traceback (most recent call last):
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [125]:
df_sexo.agg({'num_sexo': 'max'}).show()

+-------------+
|max(num_sexo)|
+-------------+
|            2|
+-------------+



In [22]:
df_sexo.where(F.col('num_sexo') > 1).count()

8088

In [127]:
df_sexo.show(5)

+--------------------+---------+--------+--------+
|                  id|     sexo|num_sexo|sexo_set|
+--------------------+---------+--------+--------+
|000009b2e3aaae800...|   [m, m]|       1|     [m]|
|00001904b82ec2994...|[m, m, m]|       1|     [m]|
|00002e4d7a02621aa...|[m, m, m]|       1|     [m]|
|000042056ba50ed6d...|   [f, f]|       1|     [f]|
|00004eadf6e493c07...|   [f, f]|       1|     [f]|
+--------------------+---------+--------+--------+
only showing top 5 rows



Traceback (most recent call last):
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [116]:
df_sexo.where(F.col('num_sexo') > 1).show(5)

+--------------------+---------+--------+--------+
|                  id|     sexo|num_sexo|sexo_set|
+--------------------+---------+--------+--------+
|02e86c1bd6e25322c...|   [m, f]|       2|  [f, m]|
|03d7e6146998df740...|[m, f, f]|       2|  [f, m]|
|0e40ce65ea5cfddaa...|   [f, m]|       2|  [f, m]|
|0fa045ad8d0bbdb01...|   [f, m]|       2|  [f, m]|
|15d2c63285b265729...|   [f, m]|       2|  [f, m]|
+--------------------+---------+--------+--------+
only showing top 5 rows



In [128]:
df_sexo = df_sexo.withColumn(
    'm',
    F.when(
        F.col('sexo_set').contains('m'), F.lit(1)
    ).otherwise(F.lit(0))
).withColumn(
    'f',
    F.when(
        F.col('sexo_set').contains('f'), F.lit(1)
    ).otherwise(F.lit(0))
).withColumn(
    'i',
    F.when(
        F.col('sexo_set').contains('i'), F.lit(1)
    ).otherwise(F.lit(0))
)

In [132]:
df_sexo.groupby('f', 'm', 'i').count().coalesce(1) \
.write.csv('tabela_sexo', header=True)

In [133]:
df_mun = spark.read.csv('mun_res', header=True)

In [136]:
df_mun.show(5, truncate=False)

+----------------------------------------------------------------+------------------------+
|id                                                              |mun_res                 |
+----------------------------------------------------------------+------------------------+
|000009b2e3aaae8009a4bc0e505720682eff8ab275c093514d6c1faf53e39b5e|[330455, 330455]        |
|00001904b82ec29942a5749c9a78ecb641f8a3a09600d21c6cfc320d009c880a|[330455, 330455, 330455]|
|00002e4d7a02621aac1b1b094f40a8d92370775f4a5df902bb97d02df682840b|[290687, 290687, 290687]|
|000042056ba50ed6de4f1a1070583935cc6177a1df0fe0bfd0633b8118f2a049|[150680, 150680]        |
|00004eadf6e493c07663ee4084ef60d3f02829ef29b5f94809c37422b676cd0c|[410370, 410370]        |
+----------------------------------------------------------------+------------------------+
only showing top 5 rows



In [137]:
df_mun.count()

161029567

In [138]:
df_mun = df_mun.withColumn(
    'num_mun',
    F.udf(len)(F.udf(set)(
        F.split(
            F.regexp_replace(F.col('mun_res'), r'^\[|\]$', ''), ', '
        )
    ))
)

In [139]:
df_mun = df_mun.withColumn(
    'mun_set', 
    F.udf(set)(F.split(
        F.regexp_replace(F.col('mun_res'), r'^\[|\]$', ''), ', '
    ))
)

In [140]:
df_mun.show(5, truncate=False)

+----------------------------------------------------------------+------------------------+-------+--------+
|id                                                              |mun_res                 |num_mun|mun_set |
+----------------------------------------------------------------+------------------------+-------+--------+
|000009b2e3aaae8009a4bc0e505720682eff8ab275c093514d6c1faf53e39b5e|[330455, 330455]        |1      |[330455]|
|00001904b82ec29942a5749c9a78ecb641f8a3a09600d21c6cfc320d009c880a|[330455, 330455, 330455]|1      |[330455]|
|00002e4d7a02621aac1b1b094f40a8d92370775f4a5df902bb97d02df682840b|[290687, 290687, 290687]|1      |[290687]|
|000042056ba50ed6de4f1a1070583935cc6177a1df0fe0bfd0633b8118f2a049|[150680, 150680]        |1      |[150680]|
|00004eadf6e493c07663ee4084ef60d3f02829ef29b5f94809c37422b676cd0c|[410370, 410370]        |1      |[410370]|
+----------------------------------------------------------------+------------------------+-------+--------+
only showing top 5 

Traceback (most recent call last):
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [141]:
df_mun.agg({'num_mun': 'max'}).show()

+------------+
|max(num_mun)|
+------------+
|           3|
+------------+



In [144]:
df_mun.where(F.col('num_mun') == 1).count()

160124557

In [145]:
df_mun.where(F.col('num_mun') == 2).count()

904606

In [146]:
df_mun.where(F.col('num_mun') == 3).count()

404

In [149]:
df_mun.select('mun_set').where(F.col('num_mun') == 3).show(5, truncate=False)

+------------------------+
|mun_set                 |
+------------------------+
|[412215, 500568, 411330]|
|[355040, 354780, 353870]|
|[170200, 520750, 500270]|
|[351380, 313210, 313520]|
|[500110, 353550, 500270]|
+------------------------+
only showing top 5 rows



In [170]:
df_mun = df_mun.withColumn(
    'mun_set_c',
    F.regexp_extract(F.col('mun_set'), r'(\d+)', 1)
)

In [171]:
df_mun.groupby('mun_set_c').count().coalesce(1) \
.write.csv('tabela_mun_res', header=True)

In [172]:
df_idade = spark.read.csv('idade', header=True)

In [174]:
df_idade.show(5)

+--------------------+------------+
|                  id|       idade|
+--------------------+------------+
|000009b2e3aaae800...|    [50, 50]|
|00001904b82ec2994...|[61, 61, 62]|
|00002e4d7a02621aa...|[69, 69, 69]|
|000042056ba50ed6d...|    [34, 34]|
|00004eadf6e493c07...|    [35, 35]|
+--------------------+------------+
only showing top 5 rows



In [175]:
df_idade = df_idade.withColumn(
    'num_idade', 
    F.udf(len)(F.udf(set)(
        F.split(F.regexp_replace(F.col('idade'), r'^\[|\]$', ''), ', ')
    ))
)

In [176]:
df_idade = df_idade.withColumn(
    'idade_set',
    F.udf(set)(
        F.split(F.regexp_replace(F.col('idade'), r'^\[|\]$', ''), ', ')
    )
)

In [177]:
df_idade.show(5)

+--------------------+------------+---------+---------+
|                  id|       idade|num_idade|idade_set|
+--------------------+------------+---------+---------+
|000009b2e3aaae800...|    [50, 50]|        1|     [50]|
|00001904b82ec2994...|[61, 61, 62]|        2| [61, 62]|
|00002e4d7a02621aa...|[69, 69, 69]|        1|     [69]|
|000042056ba50ed6d...|    [34, 34]|        1|     [34]|
|00004eadf6e493c07...|    [35, 35]|        1|     [35]|
+--------------------+------------+---------+---------+
only showing top 5 rows



In [178]:
df_idade.agg({'num_idade': 'max'}).show()

+--------------+
|max(num_idade)|
+--------------+
|             4|
+--------------+



In [179]:
df_idade = df_idade.withColumn('idade_c', F.regexp_extract(F.col('idade_set'), r'(\d+)', 1))

In [217]:
df_idade = df_idade.withColumn(
    'idade_c',
    F.col('idade_c').cast('int')
).withColumn(
    'fx_etaria',
    fx_etaria(F.col('idade_c'))
)

In [218]:
df_idade.groupby('fx_etaria').count().coalesce(1) \
.write.csv('tabela_faixa_etaria', mode='overwrite', header=True)

In [230]:
df_dose = spark.read.csv('doses', header=True)

In [193]:
df_dose.select('1a dose').where(F.col('1a dose') < '2020-01-17').count()

153

In [195]:
df_dose.select('unica').where(F.col('unica') < '2020-01-17').count()

1

In [196]:
df_dose.select('2a dose').where(F.col('2a dose') < '2020-01-17').count()

194

In [197]:
df_dose.select('reforco').where(F.col('reforco') < '2020-01-17').count()

9

In [108]:
# Essa função é uma adapatação de emergência para trabalhar os dados de Irece
def adjust_dose_date(df):
    df = df.withColumn(
        '1a dose',
        F.when(
            F.split(F.col('1a dose'), ',').getItem(0) < '2020-01-17',
            F.concat_ws(
                ',',
                F.lit('2020-01-17'),
                F.split(F.col('1a dose'), ',').getItem(1)
            )
        ).otherwise(F.col('1a dose'))
    ).withColumn(
        'unica',
        F.when(
            F.split(F.col('unica'), ',').getItem(0) < '2020-01-17',
            F.concat_ws(
                ',',
                F.lit('2020-01-17'),
                F.split(F.col('unica'), ',').getItem(1)
            )
        ).otherwise(F.col('unica'))
    ).withColumn(
        '2a dose',
        F.when(
            (F.split(F.col('2a dose'), ',').getItem(0) < '2020-01-17') &
            (F.split(F.col('1a dose'), ',').getItem(0) >= '2020-01-17'),
            F.concat_ws(
                ',',
                F.add_months(F.split(F.col('1a dose'), ',').getItem(0), 3),
                F.split(F.col('2a dose'), ',').getItem(1)
            )
        ).otherwise(F.col('2a dose'))
    ).withColumn(
        'reforco',
        F.when(
            (F.split(F.col('reforco'), ',').getItem(0) <
             F.split(F.col('2a dose'), ',').getItem(0)),
            F.concat_ws(
                ',',
                F.add_months(F.split(F.col('2a dose'), ',').getItem(0), 3),
                F.split(F.col('reforco'), ',').getItem(1)
            )
        ).otherwise(F.col('reforco'))
    )
    
    return df

In [231]:
df_dose = df_dose.withColumn(
    '1a dose',
    F.when(
        F.split(F.col('1a dose'), ',').getItem(0) < '2020-01-17',
        F.concat_ws(
            ',',
            F.lit('2020-01-17'),
            F.split(F.col('1a dose'), ',').getItem(1)
        )
    ).otherwise(F.col('1a dose'))
).withColumn(
    'unica',
    F.when(
        F.split(F.col('unica'), ',').getItem(0) < '2020-01-17',
        F.concat_ws(
            ',',
            F.lit('2020-01-17'),
            F.split(F.col('unica'), ',').getItem(1)
        )
    ).otherwise(F.col('unica'))
).withColumn(
    '2a dose',
    F.when(
        (F.split(F.col('2a dose'), ',').getItem(0) < '2020-01-17') &
        (F.split(F.col('1a dose'), ',').getItem(0) >= '2020-01-17'),
        F.concat_ws(
            ',',
            F.add_months(F.split(F.col('1a dose'), ',').getItem(0), 3),
            F.split(F.col('2a dose'), ',').getItem(1)
        )
    ).when(
        (F.split(F.col('2a dose'), ',').getItem(0) < '2020-01-17') &
        (F.split(F.col('dose inicial'), ',').getItem(0) >= '2020-01-17'),
        F.concat_ws(
            ',',
            F.add_months(F.split(F.col('dose inicial'), ',').getItem(0), 3),
            F.split(F.col('2a dose'), ',').getItem(1)
        )
    ).otherwise(F.col('2a dose'))
).withColumn(
    'reforco',
    F.when(
        (F.split(F.col('reforco'), ',').getItem(0) <
         F.split(F.col('2a dose'), ',').getItem(0)),
        F.concat_ws(
            ',',
            F.add_months(F.split(F.col('2a dose'), ',').getItem(0), 3),
            F.split(F.col('reforco'), ',').getItem(1)
        )
    ).otherwise(F.col('reforco'))
)

In [223]:
df_dose.select('1a dose', 'dose inicial', 'unica', '2a dose') \
.where(F.col('2a dose') < '2020-01-17').show(truncate=False)

+-------+------------+-----+------------+
|1a dose|dose inicial|unica|2a dose     |
+-------+------------+-----+------------+
|null   |null        |null |1995-05-23,3|
|null   |null        |null |1975-08-27,1|
|null   |null        |null |1986-02-19,3|
|null   |null        |null |2001-05-24,1|
+-------+------------+-----+------------+



In [232]:
df_dose.select('1a dose', '2a dose', 'unica', 'dose inicial', 'reforco') \
.where(F.col('reforco') < '2020-01-17').show(truncate=False)

+-------+-------+-----+------------+------------+
|1a dose|2a dose|unica|dose inicial|reforco     |
+-------+-------+-----+------------+------------+
|null   |null   |null |null        |1932-06-03,3|
+-------+-------+-----+------------+------------+



In [221]:
def generate_labels_dose(df):
    df = df.withColumn(
        'prim_dose',
        F.when(
            (F.col('2a dose').isNull()) &
            (F.col('reforco').isNull()) &
            (F.col('3a dose').isNull()),
            F.when(
                F.col('1a dose').isNotNull(),
                F.regexp_extract('1a dose', '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
            )
        )
    ).withColumn(
        'vac_s_ref',
        F.when(
            (F.col('1a dose').isNotNull()) &
            (F.col('2a dose').isNotNull()) &
            (F.col('unica').isNull()) &
                # (F.col('1o reforco').isNull()) &
            (F.col('reforco').isNull()) &
            (F.col('3a dose').isNull()),
            F.regexp_extract(F.col('2a dose'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
        ).when(
                # (F.col('dose inicial').isNotNull()) &
            (F.col('2a dose').isNotNull()) &
            (F.col('unica').isNull()) &

                # (F.col('1o reforco').isNull()) &
            (F.col('reforco').isNull()) &
            (F.col('3a dose').isNull()),
            F.regexp_extract(F.col('2a dose'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
        ).when(
            (F.col('unica').isNotNull()) &
            (F.col('1a dose').isNull()) &
                # (F.col('dose inicial').isNull()) &
            (F.col('2a dose').isNull()) &

                # (F.col('1o reforco').isNull()) &
            (F.col('reforco').isNull()) &
            (F.col('3a dose').isNull()),
            F.regexp_extract(F.col('unica'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
        )
    ).withColumn(
        'vac_c_ref',
        F.when(
            (F.col('1a dose').isNotNull()) &
            (F.col('2a dose').isNotNull()) &
            (F.col('unica').isNull()),
            F.when(
                F.col('reforco').isNotNull(), F.regexp_extract(F.col('reforco'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)

            ).when(
                F.col('3a dose').isNotNull(), F.regexp_extract(F.col('3a dose'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
            )
            ).when(
                (F.col('2a dose').isNotNull()) &
                (F.col('unica').isNull()),
                F.when(
                    F.col('reforco').isNotNull(), F.regexp_extract(F.col('reforco'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
                ).when(
                    F.col('3a dose').isNotNull(), F.regexp_extract(F.col('3a dose'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
                )
            ).when(
                (F.col('unica').isNotNull()) &
                (F.col('1a dose').isNull()) &
                (F.col('2a dose').isNull()),
                F.when(
                    F.col('reforco').isNotNull(), F.regexp_extract(F.col('reforco'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
                ).when(
                    F.col('3a dose').isNotNull(), F.regexp_extract(F.col('3a dose'), '\\d{4}-\\d{1,2}-\\d{1,2}', 0)
                )
        )
    )

    return df

In [294]:
df_dose = df_dose.withColumn(
    'prim_dose',
    F.when(
        (F.col('2a dose').isNull()) &
        (F.col('1o reforco').isNull()) &
        (F.col('reforco').isNull()) &
        (F.col('3a dose').isNull()) &
        (F.col('dose adicional').isNull()),
        F.when(
            F.col('1a dose').isNotNull(),
            (F.col('1a dose'))
        ).when(
            F.col('dose inicial').isNotNull(),
            (F.col('dose inicial'))
        ).when(
            (F.col('dose inicial').isNotNull()) &
            (F.col('1a dose').isNotNull()),
            F.col('1a dose'))
    )
).withColumn(
    'vac_s_ref',
    F.when(
        (F.col('1a dose').isNotNull()) &
        (F.col('2a dose').isNotNull()) &
        (F.col('unica').isNull()) &
        
        (F.col('1o reforco').isNull()) &
        (F.col('reforco').isNull()) &
        (F.col('3a dose').isNull()) &
        (F.col('dose adicional').isNull()),
        
        F.col('2a dose')
    ).when(
        (F.col('dose inicial').isNotNull()) &
        (F.col('2a dose').isNotNull()) &
        (F.col('unica').isNull()) &
        
        (F.col('1o reforco').isNull()) &
        (F.col('reforco').isNull()) &
        (F.col('3a dose').isNull()) &
        (F.col('dose adicional').isNull()),
        
        (F.col('2a dose'))
    ).when(
        (F.col('unica').isNotNull()) &
        (F.col('1a dose').isNull()) &
        (F.col('dose inicial').isNull()) &
        (F.col('2a dose').isNull()) &
        
        (F.col('1o reforco').isNull()) &
        (F.col('reforco').isNull()) &
        (F.col('3a dose').isNull()) &
        (F.col('dose adicional').isNull()),
        
        F.col('unica')
    )
).withColumn(
    'vac_c_ref',
    F.when(
        (F.col('1a dose').isNotNull()) &
        (F.col('2a dose').isNotNull()) &
        (F.col('unica').isNull()),
        F.when(
            F.col('1o reforco').isNotNull(), F.col('1o reforco')
        ).when(
            F.col('reforco').isNotNull(), F.col('reforco')
        ).when(
            F.col('3a dose').isNotNull(), F.col('3a dose')
        ).when(
            F.col('dose adicional').isNotNull(), F.col('dose adicional')
        )
    ).when(
        (F.col('dose inicial').isNotNull()) &
        (F.col('2a dose').isNotNull()) &
        (F.col('unica').isNull()),
        F.when(
            F.col('1o reforco').isNotNull(), F.col('1o reforco')
        ).when(
            F.col('reforco').isNotNull(), F.col('reforco')
        ).when(
            F.col('3a dose').isNotNull(), F.col('3a dose')
        ).when(
            F.col('dose adicional').isNotNull(), F.col('dose adicional')
        )
    ).when(
        (F.col('unica').isNotNull()) &
        (F.col('1a dose').isNull()) &
        (F.col('dose inicial').isNull()) &
        (F.col('2a dose').isNull()),
        F.when(
            F.col('1o reforco').isNotNull(), F.col('1o reforco')
        ).when(
            F.col('reforco').isNotNull(), F.col('reforco')
        ).when(
            F.col('3a dose').isNotNull(), F.col('3a dose')
        ).when(
            F.col('dose adicional').isNotNull(), F.col('dose adicional')
        )
    )
)

In [125]:
def rename_dose(df):
    df = df.withColumn(
        'apenas_primeira_dose_nome',
        F.when(
            F.split(F.col('prim_dose'), ',').getItem(1) == 1, 'astrazeneca'
        ).when(
            F.split(F.col('prim_dose'), ',').getItem(1) == 2, 'coronavac'
        ).when(
            F.split(F.col('prim_dose'), ',').getItem(1) == 3, 'pfizer'
        ).when(
            F.split(F.col('prim_dose'), ',').getItem(1) == 4, 'janssen'
        )
    ).withColumn(
        'vacinados_sem_reforco_nome',
        F.when(
            F.split(F.col('vac_s_ref'), ',').getItem(1) == 1, 'astrazeneca'
        ).when(
            F.split(F.col('vac_s_ref'), ',').getItem(1) == 2, 'coronavac'
        ).when(
            F.split(F.col('vac_s_ref'), ',').getItem(1) == 3, 'pfizer'
        ).when(
            F.split(F.col('vac_s_ref'), ',').getItem(1) == 4, 'janssen'
        )
    ).withColumn(
        'vacinados_com_reforco_nome',
        F.when(
            F.split(F.col('vac_c_ref'), ',').getItem(1) == 1, 'astrazeneca'
        ).when(
            F.split(F.col('vac_c_ref'), ',').getItem(1) == 2, 'coronavac'
        ).when(
            F.split(F.col('vac_c_ref'), ',').getItem(1) == 3, 'pfizer'
        ).when(
            F.split(F.col('vac_c_ref'), ',').getItem(1) == 4, 'janssen'
        )
    )
    
    return df

In [295]:
df_dose = df_dose.withColumn(
    'apenas_primeira_dose_nome',
    F.when(
        F.split(F.col('prim_dose'), ',').getItem(1) == 1, 'astrazeneca'
    ).when(
        F.split(F.col('prim_dose'), ',').getItem(1) == 2, 'coronavac'
    ).when(
        F.split(F.col('prim_dose'), ',').getItem(1) == 3, 'pfizer'
    ).when(
        F.split(F.col('prim_dose'), ',').getItem(1) == 4, 'janssen'
    )
).withColumn(
    'vacinados_sem_reforco_nome',
    F.when(
        F.split(F.col('vac_s_ref'), ',').getItem(1) == 1, 'astrazeneca'
    ).when(
        F.split(F.col('vac_s_ref'), ',').getItem(1) == 2, 'coronavac'
    ).when(
        F.split(F.col('vac_s_ref'), ',').getItem(1) == 3, 'pfizer'
    ).when(
        F.split(F.col('vac_s_ref'), ',').getItem(1) == 4, 'janssen'
    )
).withColumn(
    'vacinados_com_reforco_nome',
    F.when(
        F.split(F.col('vac_c_ref'), ',').getItem(1) == 1, 'astrazeneca'
    ).when(
        F.split(F.col('vac_c_ref'), ',').getItem(1) == 2, 'coronavac'
    ).when(
        F.split(F.col('vac_c_ref'), ',').getItem(1) == 3, 'pfizer'
    ).when(
        F.split(F.col('vac_c_ref'), ',').getItem(1) == 4, 'janssen'
    )
)

In [126]:
def p_dose_date(df):
    df = df.withColumn(
        'apenas_primeira_dose_data',
        F.split(F.col('prim_dose'), ',').getItem(0)
    ).withColumn(
        'vacinados_sem_reforco_data',
        F.split(F.col('vac_s_ref'), ',').getItem(0)
    ).withColumn(
        'vacinados_com_reforco_data',
        F.split(F.col('vac_c_ref'), ',').getItem(0)
    )
    
    return df

In [296]:
df_dose = df_dose.withColumn(
    'apenas_primeira_dose_data',
    F.split(F.col('prim_dose'), ',').getItem(0)
).withColumn(
    'vacinados_sem_reforco_data',
    F.split(F.col('vac_s_ref'), ',').getItem(0)
).withColumn(
    'vacinados_com_reforco_data',
    F.split(F.col('vac_c_ref'), ',').getItem(0)
)

In [298]:
df_dose.groupby('apenas_primeira_dose_nome',
                 'vacinados_sem_reforco_nome',
                 'vacinados_com_reforco_nome') \
.count() \
.coalesce(1) \
.write \
.csv('tabela_doses_nome', mode='overwrite', header=True)

In [299]:
df_dose.groupby('apenas_primeira_dose_data',
                 'vacinados_sem_reforco_data',
                 'vacinados_com_reforco_data') \
.count() \
.coalesce(1) \
.write \
.csv('tabela_doses_data', mode='overwrite', header=True)